In [5]:
import tensorflow as tf
import pandas as pd
import ssl

from stable_baselines.common.policies import MlpLnLstmPolicy
from stable_baselines import PPO2

from tensortrade.strategies import StableBaselinesTradingStrategy
from tensortrade.environments import TradingEnvironment
from tensortrade.rewards import RiskAdjustedReturns
from tensortrade.actions import PairCriteriaSizeActions
from tensortrade.instruments import Quantity, TradingPair, BTC, USD
from tensortrade.wallets import Wallet, Portfolio
from tensortrade.exchanges.simulated import SimulatedExchange
from tensortrade.features.stationarity import FractionalDifference
from tensortrade.features.scalers import MinMaxNormalizer
from tensortrade.features import FeaturePipeline

WINDOW_SIZE = 20
PRICE_COLUMN = 'close'

normalize = MinMaxNormalizer(inplace=True)
difference = FractionalDifference(difference_order=0.6, inplace=True)
feature_pipeline = FeaturePipeline(steps=[normalize, difference])

action_scheme = PairCriteriaSizeActions(TradingPair(USD, BTC))
reward_scheme = RiskAdjustedReturns(return_algorithm="sharpe")

csv_file = tf.keras.utils.get_file(
    'Coinbase_BTCUSD_1h.csv', 'https://www.cryptodatadownload.com/cdd/Coinbase_BTCUSD_1h.csv')
ohlcv_data = pd.read_csv(csv_file, skiprows=1, index_col="Date")
ohlcv_data.columns = map(str.lower, ohlcv_data.columns)
ohlcv_data = ohlcv_data.rename(columns={'volume btc': 'volume'})

# ohlcv_data = pd.read_csv('./data/updateOHLCdata_master.csv')
ohlcv_data = ohlcv_data.iloc[0:10000]

exchange = SimulatedExchange(base_instrument=USD,
                             data_frame=ohlcv_data,
                             price_column=PRICE_COLUMN,
                             window_size=WINDOW_SIZE,
                             pretransform=True)

exchange.reset()

quantity_usd = Quantity(USD, 10000)
quantity_btc = Quantity(BTC, 0)

wallet_usd = Wallet(exchange, quantity_usd)
wallet_btc = Wallet(exchange, quantity_btc)

portfolio = Portfolio(base_instrument=USD)

portfolio.add(wallet_usd)
portfolio.add(wallet_btc)

exchange.portfolio = portfolio

exchange.reset()

print('Transformed Data:')
print(exchange.observation_columns)

model = PPO2
policy = MlpLnLstmPolicy
params = {'learning_rate': 1e-5,
          'nminibatches': 1,
          'verbose': 2}

environment = TradingEnvironment(exchange=exchange,
                                 action_scheme=action_scheme,
                                 reward_scheme=reward_scheme,
                                 feature_pipeline=feature_pipeline)

strategy = StableBaselinesTradingStrategy(environment=environment,
                                          model=model,
                                          policy=policy,
                                          model_kwargs=params)

performance = strategy.run(steps=10000)

Exception: URL fetch failure on https://www.cryptodatadownload.com/cdd/Coinbase_BTCUSD_1h.csv: None -- [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1076)

In [1]:
import os

In [3]:
os.chdir("/Users/MatthewWBrulhardt/projects/tensortrade/")

In [ ]:
exchange = SimulatedExchange(base_instrument=USD,
                             data_frame=ohlcv_data,
                             price_column=PRICE_COLUMN,
                             window_size=WINDOW_SIZE,
                             pretransform=True)

specification = [
    (exchange, USD, 10000),
    (exchange, BTC, 0)
]

portfolio.from_tuples()